Notebook for calling DMI climateData API and process the data
=====
This however is quite a slow process (15-20 min to download ~600k rows, with our total dataset being >20 million rows), so we instead bulk download the data through https://dmigw.govcloud.dk/v2/climateData/bulk/?api-key= and process the data locally in [loadbulkweather.ipynb](loadbulkweather.ipynb).

In [1]:
import requests
import pandas as pd

import assetpricing_functions as ap
%load_ext autoreload
%autoreload 2

In [2]:
# api keys
forecast_api_key = 'e55dced6-70fb-44f1-a658-cdef62c74b6a'
climate_api_key = 'fa9056ec-2f41-4042-828b-91750e966966'
metobs_api_key = 'd40321ee-7de5-4417-a0bf-108bd34061ab'

In [3]:
# parameters
base_url = 'https://dmigw.govcloud.dk/v2/climateData/collections/20kmGridValue/items'
parameter_ids =  ['mean_temp', 'mean_wind_speed', 'mean_wind_dir','max_wind_speed_10min']  # Add more parameters if needed
period = '2024-01-01T00:00:00Z/2024-10-01T00:00:00Z' # edit if needed should be in format 'YYYY-MM-DDTHH:MM:SSZ/YYYY-MM-DDTHH:MM:SSZ'

In [4]:
# Initialize an empty DataFrame to hold the final results
df_final = pd.DataFrame()

# Loop over each parameterId
for param_id in parameter_ids:
    # Fetch all data for the current parameterId using pagination
    data = ap.dmi_api(base_url, period, param_id, api_key=climate_api_key)

    # Convert the data to a DataFrame
    df_current = pd.DataFrame(data)

    # Merge the current DataFrame with the final one
    if df_final.empty:
        df_final = df_current  # First parameter: just initialize the final DataFrame
    else:
        df_final = pd.merge(df_final, df_current, on=['cellId', 'from', 'to'], how='outer')

In [5]:
# printing df
df_final

,cellId,from,to,mean_temp,mean_wind_speed,mean_wind_dir,max_wind_speed_10min
0,20km_604_64,2024-05-13T14:00:00+00:00,2024-05-13T15:00:00+00:00,17.1,NaN,NaN,NaN
1,20km_604_64,2024-05-13T15:00:00+00:00,2024-05-13T16:00:00+00:00,16.8,7.4,88.0,8.1
2,20km_604_64,2024-05-13T16:00:00+00:00,2024-05-13T17:00:00+00:00,16.3,7.3,88.0,8.2
3,20km_604_64,2024-05-13T17:00:00+00:00,2024-05-13T18:00:00+00:00,15.4,6.7,92.0,7.5
4,20km_604_64,2024-05-13T18:00:00+00:00,2024-05-13T19:00:00+00:00,14.2,6.4,91.0,7.2
...,...,...,...,...,...,...,...
600033,20km_638_58,2024-09-30T20:00:00+00:00,2024-09-30T21:00:00+00:00,13.2,11.5,126.0,13.1
600034,20km_638_58,2024-09-30T21:00:00+00:00,2024-09-30T22:00:00+00:00,12.8,11.7,126.0,13.4
600035,20km_638_58,2024-09-30T22:00:00+00:00,2024-09-30T23:00:00+00:00,12.6,11.0,123.0,12.9
600036,20km_638_58,2024-09-30T23:00:00+00:00,2024-10-01T00:00:00+00:00,12.2,11.1,115.0,13.2


In [6]:
df_final[(df_final['cellId']=='20km_604_64')]

,cellId,from,to,mean_temp,mean_wind_speed,mean_wind_dir,max_wind_speed_10min
0,20km_604_64,2024-05-13T14:00:00+00:00,2024-05-13T15:00:00+00:00,17.1,NaN,NaN,NaN
1,20km_604_64,2024-05-13T15:00:00+00:00,2024-05-13T16:00:00+00:00,16.8,7.4,88.0,8.1
2,20km_604_64,2024-05-13T16:00:00+00:00,2024-05-13T17:00:00+00:00,16.3,7.3,88.0,8.2
3,20km_604_64,2024-05-13T17:00:00+00:00,2024-05-13T18:00:00+00:00,15.4,6.7,92.0,7.5
4,20km_604_64,2024-05-13T18:00:00+00:00,2024-05-13T19:00:00+00:00,14.2,6.4,91.0,7.2
...,...,...,...,...,...,...,...
3366,20km_604_64,2024-09-30T20:00:00+00:00,2024-09-30T21:00:00+00:00,12.0,9.3,116.0,10.4
3367,20km_604_64,2024-09-30T21:00:00+00:00,2024-09-30T22:00:00+00:00,11.8,9.4,116.0,10.7
3368,20km_604_64,2024-09-30T22:00:00+00:00,2024-09-30T23:00:00+00:00,11.7,9.3,115.0,10.7
3369,20km_604_64,2024-09-30T23:00:00+00:00,2024-10-01T00:00:00+00:00,11.4,9.3,114.0,10.6
